<h3>Guided Acivity 4.2 Scara with Prismatic Manipulator</h3>

In this Jupyter Notebook we are going to model a Scara Robot with a Prismatic Manipulator and as a final exercise you are going to make a connection to the Coppelia (this part is what we have done in class in guided activity 4.1)

In [ ]:
# You have a Guided Video to help you perform exercise 4.1

from IPython.display import YouTubeVideo

YouTubeVideo('uTJgaM2UJSY') 

The image in the figure represents a SCARA robot studied in class and its D-H table.<br>
<img src="scara.jpg" width="500"><br>
From the parameters indicated in the image, we calculate the coordinate transformation matrices for each joint.

In [ ]:
# We import the libraries necessary to work
import numpy as np
import sympy as sp
from sympy import *

In [ ]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

In [ ]:
# we declare the symbols (lengths, variables, etc.) that will be used for a later formulation.

from sympy.physics.mechanics import dynamicsymbols
theta1, theta2, d3, lc, la, lb, theta, alpha, a, d = dynamicsymbols('theta1 theta2 d3 lc la lb theta alpha a d')
theta1, theta2, d3, lc, la, lb, theta, alpha, a, d 

In [ ]:
# The standard homogeneous transformation matrix is represented as:

rot = sp.Matrix([[sp.cos(theta), -sp.sin(theta)*sp.cos(alpha), sp.sin(theta)*sp.sin(alpha)],
                 [sp.sin(theta), sp.cos(theta)*sp.cos(alpha), -sp.cos(theta)*sp.sin(alpha)],
                 [0, sp.sin(alpha), sp.cos(alpha)]])

trans = sp.Matrix([a*sp.cos(theta),a*sp.sin(theta),d])

last_row = sp.Matrix([[0, 0, 0, 1]])

m = sp.Matrix.vstack(sp.Matrix.hstack(rot, trans), last_row)
m

In [ ]:
# Transformation: 1st axis from '0' to '1'
m01 = m.subs({ theta:theta1, d:lc, a:la , alpha:0})
m01

In [ ]:
# Transformation: 2nd axis from '1' to '2'
# The 180° of Alpha must be converted to radians
# problem with rounding to 0  http://research.iac.es/sieinvens/python-course/source/sympy.html
m12  = m.subs({ theta:theta2, d:0,a:lb ,alpha:180*np.pi/180})
N(m12,2)

In [ ]:
# We solve Sympy's problem of rounding to 0 by directly assigning the value
m12[0,2]=0
m12[1,2]=0
m12[2,1]=0  #remember the problem when rounding to 0 http://research.iac.es/sieinvens/python-course/source/sympy.html
m12

In [ ]:
# Transformation: 3rd axis from '2' to '3'
m23 = m.subs({ theta:0, d:d3, a:0 ,alpha:0})
m23

In [ ]:
# Transformed Matrix of the Robot without simplifying
m03 = (m01*m12*m23)
m03

The result is the same given in theory:<br>
<img src="mbeescara.jpg" width="500"><br>

In [ ]:
# We can simplify it more:
mbee= sp.Matrix([[sp.trigsimp(m03[0,0].simplify()), sp.trigsimp(m03[0,1].simplify()), sp.trigsimp(m03[0,2].simplify()),sp.trigsimp(m03[0,3].simplify())],
                 [sp.trigsimp(m03[1,0].simplify()), sp.trigsimp(m03[1,1].simplify()), sp.trigsimp(m03[1,2].simplify()),sp.trigsimp(m03[1,3].simplify())],
                 [m03[2,0].simplify(), m03[2,1].simplify(), m03[2,2].simplify(),m03[2,3].simplify()],
                 [m03[3,0].simplify(), m03[3,1].simplify(), m03[3,2].simplify(),m03[3,3].simplify()]])

mbee
    

##  Checking that the values   (Forward Kinematics)

In [ ]:
# We can check if it works by substituting some parameters to the robot values in Coppelia :
# The lengths of the robot made in copelia are la:0.2,lb:0.2,   lc:0.1 (height of Robot)
# Si por ejemplo ponemos que Theta1 son 0grados y Theta2 son 90grados y la d3 son 0.05
mbee = mbee.subs({ theta1:0*np.pi/180, theta2:90*np.pi/180, la:0.2,lb:0.2,lc:0.1,d3:0.05})
N(mbee,4)
# Nos ha de dar la posición del manipulador a (0.2,0.2,0.05) 

## Now we are going to determine the Robot Work AREA, we will use the Plotly library.
### This representation must be done in layers, initially we will assume the position of d3=0.1, thus we will draw the layer Z=0 and then d3=0 representing the layer Z=0.1


In [ ]:
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) # for offline mode in Jupyter Notebook use
import plotly.graph_objs as go
from numpy import * 
from plotly.subplots import make_subplots

la, lb, lc = 0.2,0.2,0.1  # arm length and height values
theta1, theta2 = np.mgrid[-np.pi/2:np.pi/2:100j, -np.pi/2:np.pi/2:100j] # motors that can move from -90 to 90º

x = la*cos(theta1) + lb*cos(theta1 + theta2)
y = la*sin(theta1) + lb*sin(theta1 + theta2)
z = y*0   # The initial area represented is in the layer Z=0 d3=0.1 --> Z=0.1-d3=0

fig = make_subplots(rows=1, cols=1,
                    specs=[[{'is_3d': True}]],
                    subplot_titles=['WORK AREA FOR LAYERS d3=0.1 and d3=0'],
                    )

fig.add_trace(go.Surface(x=x, y=y, z=z, colorbar_x=-0.07))
# The final area represented is in layer Z=0.1,d3=0 Z=0.1-d3=0.1
fig.add_trace(go.Surface(x=x, y=y, z=z+0.1, colorbar_x=-0.07)) 
fig.update_layout(title_text="SCARA WITH PRISMATIC MANIPULATOR")
fig.show()

##  Final exercise you are going to make a connection to the Coppelia using the API SIM (this part is what we have done in class in guided activity 4.1).




### Import libraries

In [ ]:
import coppeliasim_zmqremoteapi_client
from coppeliasim_zmqremoteapi_client import RemoteAPIClient


### 1. Establish the connection

In [ ]:
def connect(port=23000):
    try:
        client = RemoteAPIClient('127.0.0.1', port)
        sim = client.getObject('sim') # Careful! function is getObject now (capital O) 
        print(f"Connected to CoppeliaSim on port {port}")
        return sim
    except Exception as e:
        print(f"Could not connect to CoppeliaSim on port {port}: {e}")
        return None

sim = connect()
if sim:
    print("Simulation time:", sim.getSimulationTime())

<h3> 2. Get the handlers </h3>

In [ ]:
dummy = sim.getObject('/FinalEffector')
print(f"Dummy handle: {dummy}")

In [ ]:
pos = sim.getObjectPosition(dummy, -1)
print(pos)

<h3> 3. What is the position of the joints?</h3>

In [ ]:
#New ZeroMQ code
joint1 = sim.getObject('/joint1')
joint2 = sim.getObject('/joint2')
print("Joint handles:", joint1, joint2)

In [ ]:
angle1 = sim.getJointPosition(joint1)
angle2 = sim.getJointPosition(joint2)

print(f"Joint 1 angle: {angle1:.2f} rad")
print(f"Joint 2 angle: {angle2:.2f} rad")

<h3> 4. We move the robot  p.e. theta1=45  theta2=45 d3=0.07 </h3>

In [ ]:
q1 = 45 * np.pi / 180  # convert to radians
sim.setJointPosition(joint1, q1)
print("Joint 1 target position set to", q1)

In [ ]:
q2 = 45 * np.pi / 180
sim.setJointPosition(joint2, q2)
print("Joint 1 target position set to", q2)

In [ ]:
pjoint = sim.getObject('/Prismatic_joint')
print("Prismatic joint handle:", pjoint)

In [ ]:
sim.setJointPosition(pjoint, 0.07)

In [ ]:
pos = sim.getObjectPosition(dummy, -1)
print(pos)